# Load Libraries

In [1]:
!TF_ENABLE_ONEDNN_OPTS=0
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2025-10-01 22:35:18.252048: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-01 22:35:18.318791: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-01 22:35:21.246829: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Num GPUs Available:  0


W0000 00:00:1759372523.186024  361067 gpu_device.cc:2342] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
import warnings
warnings.filterwarnings('ignore')

import glob
import io
import datasets
import os
import time
import joblib
import json
import csv
import pathlib
import librosa
import librosa.display

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import xgboost as xgb
import seaborn as sns

from tqdm.notebook import tqdm
from PIL import Image
from matplotlib import pyplot
from collections import Counter
from pprint import pprint
from pydub import AudioSegment # sudo apt install ffmpeg
%matplotlib inline

# Preprocessing
from sklearn.preprocessing import (
    LabelEncoder, 
    StandardScaler,
    MinMaxScaler,
    scale
    )
from sklearn.model_selection import (
    GridSearchCV, 
    train_test_split, 
    RepeatedStratifiedKFold, 
    cross_val_score, 
    KFold,
    StratifiedKFold
    ) 
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    roc_curve,
    roc_auc_score, 
    precision_recall_curve,
    auc,
    precision_score, 
    recall_score, 
    f1_score
    )


import torch.utils.data
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import load_dataset, DatasetDict, Audio

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report, accuracy_score

from transformers import WhisperModel, WhisperFeatureExtractor, AdamW
# from transformers import WhisperEncoder
from transformers import WhisperProcessor

from functions_whisper_model import SpeechClassificationDataset, SpeechClassifier, train, evaluate

In [4]:
list_datasets = [
    # ['fsdkaggle'],    # 2% cough Counter({0: 1570, 1: 30})
    # ['virufy'],       # 100% cough Counter({1: 121})
    # ['esc50'],        # 2% cough Counter({0: 1960, 1: 40})
    # ['coughvid'],     # 30% cough Counter({1: 19777, 0: 10267})
    # ['coswara'],      # 25% cough Counter({0: 18914, 1: 5408})
    ['coswara', 'coughvid', 'esc50', 'fsdkaggle', 'virufy'], 
]

# Main

In [ ]:
for window_length in [
    0.1, 0.2, 0.3, 
    0.5, 0.7, 1
    ]:
    df_results = []
    for datasets_name in list_datasets:
        datasets_name.sort()
        print('')
        print('#'*60)
        print(', '.join(datasets_name))
        print(f'Window Length: {window_length}')
        print('#'*60)
        
        dataset_str = '_'.join(datasets_name)

        if not os.path.exists(f'Results_Onset/Model_Whisper_Onset/{dataset_str}'):
            os.makedirs(f'Results_Onset/Model_Whisper_Onset/{dataset_str}')
            
        path_model_save = f'Results_Onset/Model_Whisper_Onset/{dataset_str}/whisper_best_model_{window_length}s.pt'

        ################################################################
        # Load Data
        ################################################################
        df_all = pd.DataFrame()
        for dataset in datasets_name:
            df = pd.read_csv(f'Results_Onset/Sliced_Wav_Onset/dataset_{dataset}_{window_length}s_onset.csv')
            df_all = pd.concat([df_all, df], axis=0)
        df_all = df_all.reset_index(drop=True)

        ################################################################
        # Prepare Data
        ################################################################        
        df_all['filepath'] = '/home/l083319/Cough_Related/' + df_all['filepath']
        df_all = df_all[df_all['mean_amplitude'] > 0.005].reset_index(drop=True)

        for col in ['prob', 'status', 'age', 'Unnamed: 0', 'gender', 'mean_amplitude']:
            if col in df_all.columns:
                df_all = df_all.drop([col], axis=1)
        
        audio_df = df_all.rename(columns={
            'label': 'classID', 
            'filepath': 'full_path',
        })
        
        print(audio_df.shape)
        # audio_df = audio_df.sample(frac=1).groupby('classID').head(1000).reset_index(drop=True)
        audio_df = audio_df.sample(frac=1).groupby('classID').head(2000).reset_index(drop=True)
        
        # print(audio_df)
        print(Counter(audio_df['dataset']))
        print(Counter(audio_df['classID']))
        
        train_df, temp_df = train_test_split(audio_df, test_size=0.3, random_state=42, stratify=audio_df['classID'])
        val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['classID'])
        
        print('Train:', len(train_df))
        print('Val  :', len(val_df))
        print('Test :', len(test_df))
        
        train_audio_dataset = datasets.Dataset.from_dict({
            "audio": train_df["full_path"].tolist(),
            "labels": train_df["classID"].tolist()    
            }).cast_column("audio", Audio(sampling_rate=16_000))
        
        test_audio_dataset = datasets.Dataset.from_dict({
            "audio": test_df["full_path"].tolist(),
            "labels": test_df["classID"].tolist()
            }).cast_column("audio", Audio(sampling_rate=16_000))
        
        val_audio_dataset = datasets.Dataset.from_dict({
            "audio": val_df["full_path"].tolist(),
            "labels": val_df["classID"].tolist()
            }).cast_column("audio", Audio(sampling_rate=16_000))

        ################################################################
        # Load Whisper
        ################################################################
        model_checkpoint = "openai/whisper-base"
        processor = WhisperProcessor.from_pretrained(model_checkpoint)
        whisper_model = WhisperModel.from_pretrained("openai/whisper-base")
        encoder = whisper_model.encoder  # this is the encoder module
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        train_dataset = SpeechClassificationDataset(train_audio_dataset, processor, encoder)
        test_dataset = SpeechClassificationDataset(test_audio_dataset, processor, encoder)
        val_dataset = SpeechClassificationDataset(val_audio_dataset, processor, encoder)
        
        batch_size = 32
        num_labels = 2
        num_epochs = 10
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
        
        model = SpeechClassifier(num_labels, encoder).to(device)
        optimizer = AdamW(model.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-08)
        # criterion = nn.CrossEntropyLoss()
    
        class_weights = torch.tensor([2.0, 1.0]).to(device)  # weight for class 0 and class 1
        criterion = nn.CrossEntropyLoss(weight=class_weights)

        # state_dict = torch.load('/home/l083319/Cough_Related/Results/Model/whisper_best_model.pt')
        # encoder = WhisperModel.from_pretrained(model_checkpoint)
        # device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        # num_labels = 2
        # model = SpeechClassifier(num_labels, encoder).to(device)
        # model.load_state_dict(state_dict)

        ################################################################
        # Train Whisper
        ################################################################
        start = time.time() 
        train(model, train_loader, val_loader, optimizer, criterion, device, num_epochs, path_model_save)
        end = time.time() 
        
        print(f"Total runtime of the program is {round(end - start, 3)}s") 
        print('Training Done!')

        ################################################################
        # Test Whisper
        ################################################################
        print('Load Whisper Model')

        # Create a new instance of the model and load the state dictionary
        num_labels = 2
        state_dict = torch.load(path_model_save)
        model = SpeechClassifier(num_labels, encoder).to(device)
        model.load_state_dict(state_dict)
        
        print('Evaluate Data')
        _, _, _, all_labels, all_preds, all_probs = evaluate(model, test_loader, optimizer, criterion, device)
        
        print(classification_report(all_labels, all_preds))
        print('ACC:', accuracy_score(all_labels, all_preds))
        print('Training Done!')
    
        y_test = all_labels
        y_predict = all_preds
        
        acc = accuracy_score(y_test, y_predict)
        cm = confusion_matrix(y_test, y_predict)
        print(cm)
        
        lr_fpr, lr_tpr, _ = roc_curve(y_test, all_probs[:,1])
        roc_auc = auc(lr_fpr, lr_tpr)
        precision, recall, _ = precision_recall_curve(y_test, all_probs[:,1])
        pr_auc = auc(recall, precision)
        
        pre = precision_score(y_test, y_predict)
        rec = recall_score(y_test, y_predict)
        f1 = f1_score(y_test, y_predict)
        tn, fp, fn, tp = confusion_matrix(y_test, y_predict).ravel()
        spe = tn / (tn + fp)
        sen = rec
        
        columns = ['dataset', 'dataset_counter', 'label_count', 'window_length',
                   'model',
                   'acc', 'sen', 'spe', 'pre', 'rec', 'f1', 'auc', 'auprc', 'cm']  
        
        results = [[
            dataset_str,
            Counter(audio_df['dataset']),
            Counter(audio_df['classID']),
            window_length, 'Whisper',
            acc, sen, spe, pre, rec, f1,
            roc_auc, pr_auc, cm]]

        df_results.append(results)
    
        test_df['pred'] = all_preds
        test_df.to_csv(f'Results_Onset/Model_Whisper_Onset/{dataset_str}/results_test_data_{window_length}s.csv', index=False)
        
        # Check which data is predicted wrongly
        test_df_wrong = test_df[test_df['classID'] != test_df['pred']]

    df_results = pd.DataFrame(results, columns=columns)
    df_results.to_csv(f'Results_Onset/Model_Whisper_Onset/{dataset_str}/results_summary_{window_length}s.csv', index=False)
    
print('All Done!')


############################################################
coswara, coughvid, esc50, fsdkaggle, virufy
Window Length: 0.1
############################################################
(37739, 6)
Counter({'coswara': 1444, 'coughvid': 1353, 'fsdkaggle': 662, 'esc50': 420, 'virufy': 121})
Counter({0: 2000, 1: 2000})
Train: 2800
Val  : 600
Test : 600
Epoch 1/10, Batch 20/88, Train Loss: 0.6270, Run-time: 103.67s
Epoch 1/10, Batch 40/88, Train Loss: 0.5272, Run-time: 102.832s
Epoch 1/10, Batch 60/88, Train Loss: 0.4804, Run-time: 102.72s
Epoch 1/10, Batch 80/88, Train Loss: 0.3824, Run-time: 102.803s


Evaluating:   0%|          | 0/19 [00:00<?, ?it/s]

Epoch 1/10, Val Loss: 0.4448, Val Accuracy: 0.7883, Val F1: 0.7827, Best Accuracy: 0.7883
Epoch 2/10, Batch 20/88, Train Loss: 0.3248, Run-time: 103.286s
Epoch 2/10, Batch 40/88, Train Loss: 0.5226, Run-time: 102.78s
Epoch 2/10, Batch 60/88, Train Loss: 0.5056, Run-time: 102.76s
Epoch 2/10, Batch 80/88, Train Loss: 0.3415, Run-time: 102.84s


Evaluating:   0%|          | 0/19 [00:00<?, ?it/s]

Epoch 2/10, Val Loss: 0.4398, Val Accuracy: 0.7917, Val F1: 0.7874, Best Accuracy: 0.7917
Epoch 3/10, Batch 20/88, Train Loss: 0.5766, Run-time: 102.782s
Epoch 3/10, Batch 40/88, Train Loss: 0.4363, Run-time: 102.743s
Epoch 3/10, Batch 60/88, Train Loss: 0.4093, Run-time: 102.827s
Epoch 3/10, Batch 80/88, Train Loss: 0.3084, Run-time: 102.752s


...

Epoch 7/10, Val Loss: 0.4292, Val Accuracy: 0.8067, Val F1: 0.8048, Best Accuracy: 0.8250
Epoch 8/10, Batch 20/88, Train Loss: 0.3098, Run-time: 102.916s
Epoch 8/10, Batch 40/88, Train Loss: 0.3943, Run-time: 102.885s
Epoch 8/10, Batch 60/88, Train Loss: 0.2020, Run-time: 102.872s
Epoch 8/10, Batch 80/88, Train Loss: 0.3963, Run-time: 102.838s


Evaluating:   0%|          | 0/19 [00:00<?, ?it/s]

Epoch 8/10, Val Loss: 0.4198, Val Accuracy: 0.7867, Val F1: 0.7860, Best Accuracy: 0.8250
Epoch 9/10, Batch 20/88, Train Loss: 0.2028, Run-time: 102.987s
Epoch 9/10, Batch 40/88, Train Loss: 0.4237, Run-time: 102.892s
Epoch 9/10, Batch 60/88, Train Loss: 0.3757, Run-time: 102.879s
Epoch 9/10, Batch 80/88, Train Loss: 0.2683, Run-time: 102.967s


Evaluating:   0%|          | 0/19 [00:00<?, ?it/s]

Epoch 9/10, Val Loss: 0.4244, Val Accuracy: 0.7333, Val F1: 0.7325, Best Accuracy: 0.8250
Epoch 10/10, Batch 20/88, Train Loss: 0.2385, Run-time: 102.643s
Epoch 10/10, Batch 40/88, Train Loss: 0.2403, Run-time: 102.681s
Epoch 10/10, Batch 60/88, Train Loss: 0.3421, Run-time: 102.765s
Epoch 10/10, Batch 80/88, Train Loss: 0.3298, Run-time: 102.607s


In [ ]:
# Create an empty DataFrame to hold the combined data
combined_df = pd.DataFrame()

for window_length in [0.1, 0.2, 0.3, 0.5, 0.7, 1]:
    df_results = []
    for datasets_name in list_datasets:
        datasets_name.sort()
        
        dataset_str = '_'.join(datasets_name)
    
        df = pd.read_csv(f'Results_Onset/Model_Whisper_Onset/{dataset_str}/results_summary_{window_length}s.csv')
        combined_df = pd.concat([combined_df, df], ignore_index=True)

# Display or save the result
print(combined_df)
combined_df.to_csv(f'Results_Onset/Model_Whisper_Onset/{dataset_str}/results_summary_All.csv')